In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
# ------------------------------------------------------------------------------



app = dash.Dash(__name__)

app.css.append_css({
    "external_url":'/assets/stylesheet.css'
})

theme =  {
    'dark': True,
    'detail': '#007439',
    'primary': '#00EA64',
    'secondary': '#6E6E6E',
}


# ---------- Import and clean data (importing csv into pandas)
df = pd.read_csv("intro_bees.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Other/Dash_Introduction/intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:15])

# ------------------------------------------------------------------------------

      State  ANSI Affected by  Year state_code  Pct of Colonies Impacted
0   Alabama     1     Disease  2015         AL                     0.050
1   Alabama     1     Disease  2016         AL                     1.200
2   Alabama     1     Disease  2017         AL                     2.250
3   Alabama     1     Disease  2018         AL                     1.300
4   Alabama     1     Disease  2019         AL                     1.800
5   Alabama     1       Other  2015         AL                     4.325
6   Alabama     1       Other  2016         AL                     4.275
7   Alabama     1       Other  2017         AL                     6.325
8   Alabama     1       Other  2018         AL                     7.300
9   Alabama     1       Other  2019         AL                     3.100
10  Alabama     1  Pesticides  2015         AL                     1.825
11  Alabama     1  Pesticides  2016         AL                     4.000
12  Alabama     1  Pesticides  2017         AL     

In [3]:
# ------------------------------------------------------------------------------
# App layout
# app.layout = html.Div([

#     html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

#     dcc.Dropdown(id="slct_year",
#                  options=[
#                      {"label": "2015", "value": 2015},
#                      {"label": "2016", "value": 2016},
#                      {"label": "2017", "value": 2017},
#                      {"label": "2018", "value": 2018}],
#                  multi=False,
#                  value=2015,
#                  style={'width': "40%"}
#                  ),

#     html.Div()
#     html.Div(id='output_container', children=[]),
#     html.Br(),
#     dcc.Graph(id='my_bee_map', figure={}),
#     html.Br(),
#     dcc.Graph(id='my_bee_map2', figure={})

# ])

app.layout = html.Div(className='row', children=[

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),
    
    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),
    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Dropdown(id="slct_disease",
                 options=[
                     {"label": "Pesticides", "value": 'Pesticides'},
                     {"label": "Disease", "value": 'Disease'},
                     {"label": "Pests excl Var", "value": 'Pests_excl_Varroa'},
                     {"label": "Varroa mites", "value": 'Varroa_mites'}],
                 multi=False,
                 value='Disease',
                 style={'width': "40%"}
                 ),
    html.Div(id='output_container2', children=[]),
    html.Br(),
    html.Div(children=[
        dcc.Graph(id='my_bee_map', figure={}, style={'display': 'inline-block'}),
        dcc.Graph(id='my_bee_map2', figure={}, style={'display': 'inline-block'}),
        dcc.Graph(id='my_bee_map3', figure={}, style={'display': 'inline-block'})
    ]),
  
])
# ------------------------------------------------------------------------------

In [4]:
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='output_container2', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure'),
     Output(component_id='my_bee_map2', component_property='figure'),
     Output(component_id='my_bee_map3', component_property='figure')],
    [Input(component_id='slct_year', component_property='value'),
     Input(component_id='slct_disease', component_property='value')]
)
def update_graph(option_slctd,option_disease):
    print(option_slctd)
    print(type(option_slctd))
    print(option_disease)
    print(type(option_disease))

    container = "The year chosen by user was: {}".format(option_slctd)
    container2 = "The disease chosen: {}".format(option_disease)

    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == option_disease]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )
  
    # Plotly Express
    fig_2 = px.bar(
        data_frame=dff,
        x = dff['State'],
        y = dff['Pct of Colonies Impacted'],
        template='plotly_dark'
    )
    
    dff = df.copy()
    dff = dff[dff["Affected by"] == option_disease]
    dff = dff[(dff["State"] == "Idaho") | (dff["State"] == "New York") | (dff["State"] == "New Mexico")]

    fig_3 = px.line(
        data_frame=dff,
        x = dff['Year'],
        y = dff['Pct of Colonies Impacted'],
        template='plotly_dark',
        title = 'Dynamic Disease Plot',
        color ='State',
        hover_name= 'State'
    )

#     Plotly Graph Objects (GO)
#     fig = go.Figure(
#         data=[go.Choropleth(
#             locationmode='USA-states',
#             locations=dff['state_code'],
#             z=dff["Pct of Colonies Impacted"].astype(float),
#             colorscale='Reds',
#         )]
#     )
    
#     fig.update_layout(
#         title_text="Bees Affected by Mites in the USA",
#         title_xanchor="center",
#         title_font=dict(size=24),
#         title_x=0.5,
#         geo=dict(scope='usa'),
#     )

    return container, container2, fig, fig_2, fig_3


# ------------------------------------------------------------------------------

In [ ]:
# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\SATEX\Anaconda3\lib\site-packages\dash\resources.py:72: UserWarning: You have set your config to `serve_locally=True` but A local version of /assets/stylesheet.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources
  ).format(s["external_url"])


2015
<class 'int'>
Disease
<class 'str'>
2016
<class 'int'>
Disease
<class 'str'>
2016
<class 'int'>
None
<class 'NoneType'>
2016
<class 'int'>
Disease
<class 'str'>


In [ ]:
dff = df.copy()
print(dff)

In [ ]:
(39.55*13 + 5*32.42)/(18)

In [ ]:
5*

In [ ]:
(39.55*13 + 5*32.42)/(18)